# MTCNN Detector

In [1]:
import numpy as np
import tensorflow as tf
import cv2, h5py, os, sys
from keras.layers import Conv2D, MaxPool2D, Input, Activation, BatchNormalization, Dense, Flatten
from keras.layers.advanced_activations import PReLU
from keras.models import Model, Sequential
from MTCNN import create_Onet, create_Pnet, create_Rnet
from mtcnn_utils import *

Using TensorFlow backend.


## 1. Load the pre-trained weights 

Load the pre-trained weights through the `create_Pnet`, `create_Rnet`, `create_Onet` 

In [ ]:
P_Net = create_Pnet("model weights/12net.h5")
R_Net = create_Rnet("model weights/24net.h5")
O_Net = create_Onet("model weights/48net.h5")

## 2. Face Detection 
1. Define a function which loads the image and pre-processes 

2. Run the `P_Net` and `detect_face_12net`

3. Run the `R_Net` and `detect_face_24net`

4. Run the `O_Net` and `detect_face_48net`

**Notation:**
- Notice the *channels order* when pass the parameters.

- Notice the meaning of the parameters of the function `detect_face_12net`,`detect_face_24net`,`detect_face_48net`

In [3]:
def face_detection(img, threshold):
    # imgae Normalization
    norm_img = (img.copy() - 127.5) / 128                     # imgae pixels range (0, 255)
    origin_H, origin_W, origin_C = norm_img.shape
    
    sacles = calculate_scales(norm_img)
    
    ### Step 1 : Run P-Net and post process
    out = []
    for sacle in sacles:
        temp_H = int(origin_H * sacle)
        temp_W = int(origin_W * sacle)
        sacle_img = cv2.resize(norm_img, (temp_W, temp_H))     # resize the image
#         print(temp_H, temp_W)
        inputs = sacle_img.reshape(1, *sacle_img.shape)        # add a dimension to keep the NCHW(m, n_C, n_H, n_W)
        output = P_Net.predict(inputs)
        out.append(output)
        
        rectangles = []                         # define a list to store the output of "detect_face_12net"
    for i in range (len(sacles)):
        # i = #scale, first 0 select cls score, second 0 = batchnum
        cls_prob = out[i][0][0][:, :, 1]        # the confidence of the output, which means the probablity of the face at the temp_window
        roi = out[i][1][0]                      # the position of the output
        out_h, out_w = cls_prob.shape
        out_side = max(out_h, out_w)
        
        # dimensions change for better computation
        cls_prob = np.swapaxes(cls_prob, 0, 1)
        roi = np.swapaxes(roi, 0, 2)
#         print(cls_prob.shape)
        
        rectangle = detect_face_12net(cls_prob, roi, out_side, 1/sacles[i], origin_W, origin_H, threshold[0])
        rectangles.extend(rectangle)
        
    rectangles = non_max_suppression(rectangles, 0.6)
    print("P-Net rectangles shape = " + str(np.shape(rectangles)))
    
    if len(rectangles) == 0:
        return rectangles
    
    
    ### Step 2: Run R-Net and post process
    predict_24_batch = []
    R_net_out = []
    for rectangle in  rectangles:
        # crop the image from the initial norm image
        crop_img = norm_img[int(rectangle[1]): int(rectangle[3]), int(rectangle[0]): int(rectangle[2])]
        sacle_img = cv2.resize(crop_img, (24, 24))
        predict_24_batch.append(sacle_img)        # store the all crop imgaes into a list
        
    predict_24_batch = np.array(predict_24_batch)
    R_net_out = R_Net.predict(predict_24_batch)   # 'R_net_out' is a list
    
    rnet_cls_prob = R_net_out[0]                  # R-Net output is [classifier, bbox_regress]  
    rnet_cls_prob = np.array(rnet_cls_prob)
    rnet_roi = R_net_out[1]
    rnet_roi = np.array(rnet_roi)
    
    rectangles = filter_face_24net(rnet_cls_prob, rnet_roi, rectangles, origin_W, origin_H, threshold[1])
    print("R-Net rectangles shape = " + str(np.shape(rectangles)))
    
    if len(rectangles) == 0:
        return rectangles
    
    ### Step 3: Run O-Net and post process
    predict_48_batch = []
    O_net_out = []
    for rectangle in rectangles:
        crop_img = norm_img[int(rectangle[1]): int(rectangle[3]), int(rectangle[0]): int(rectangle[2])]
        sacle_img = cv2.resize(crop_img, (48, 48))
        predict_48_batch.append(sacle_img)
        
    predict_48_batch = np.array(predict_48_batch)
    O_net_out = O_Net.predict(predict_48_batch)
    
    # O-Net output is [classifier, bbox_regress, landmark_regress]
    onet_cls_prob = O_net_out[0]
#     onet_cls_prob = np.array(onet_cls_prob)
    onet_roi = O_net_out[1]
#     onet_roi = np.array(onet_roi)
    onet_pts = O_net_out[2]
#     onet_pts = np.array(onet_pts)
    
    rectangles = filter_face_48net(onet_cls_prob, onet_roi, onet_pts, rectangles, origin_W, origin_H, threshold[2])
    print("O-Net rectangles shape = " + str(np.shape(rectangles)))
    
    return rectangles

In [4]:
img = cv2.imread("image/test_image.jpeg")
threshold = [0.5, 0.6, 0.7]

In [5]:
rectangles = face_detection(img, threshold)

P-Net rectangles shape = (942, 5)
R-Net rectangles shape = (226, 5)
O-Net rectangles shape = (10, 15)


## 3. Display the image

In [6]:
draw = img.copy()
for rectangle in rectangles:
    if rectangle is not None:
        W = -int(rectangle[0]) + int(rectangle[2])
        H = -int(rectangle[1]) + int(rectangle[3])
        padding_H = 0.01 * W
        padding_W = 0.01 * H
        crop_img = img[int(rectangle[1]+padding_H):int(rectangle[3]-padding_H), int(rectangle[0]-padding_W):int(rectangle[2]+padding_W)]
        crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)
        if crop_img is None:
            continue
        if crop_img.shape[0] < 0 or crop_img.shape[1] < 0:
            continue
        cv2.rectangle(draw, (int(rectangle[0]), int(rectangle[1])), (int(rectangle[2]), int(rectangle[3])), (255, 0, 0), 1)

        for i in range(5, 15, 2):
            cv2.circle(draw, (int(rectangle[i + 0]), int(rectangle[i + 1])), 2, (0, 255, 0))


cv2.imwrite("image/out/test.jpg", draw)                       


True

In [7]:
mayday = cv2.imread("image/test_image2.jpeg")
threshold = [0.6, 0.6, 0.7]
rectangles = face_detection(mayday, threshold)
draw = mayday.copy()

for rectangle in rectangles:
    if rectangle is not None:
        W = -int(rectangle[0]) + int(rectangle[2])
        H = -int(rectangle[1]) + int(rectangle[3])
        padding_H = 0.01 * W
        padding_W = 0.01 * H
        crop_img = img[int(rectangle[1]+padding_H):int(rectangle[3]-padding_H), int(rectangle[0]-padding_W):int(rectangle[2]+padding_W)]
#         crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)
        if crop_img is None:
            continue
        if crop_img.shape[0] < 0 or crop_img.shape[1] < 0:
            continue
        cv2.rectangle(draw, (int(rectangle[0]), int(rectangle[1])), (int(rectangle[2]), int(rectangle[3])), (255, 0, 0), 1)

        for i in range(5, 15, 2):
            cv2.circle(draw, (int(rectangle[i + 0]), int(rectangle[i + 1])), 2, (0, 255, 0))


cv2.imwrite("image/out/testing.jpg", draw)                       

c = cv2.waitKey(0)

P-Net rectangles shape = (1713, 5)
R-Net rectangles shape = (246, 5)
O-Net rectangles shape = (10, 15)


True

In [8]:
my_image = cv2.imread("image/timg.jpeg")
threshold = [0.4, 0.6, 0.7]
rectangles = face_detection(my_image, threshold)
draw = my_image.copy()

for rectangle in rectangles:
    if rectangle is not None:
        W = -int(rectangle[0]) + int(rectangle[2])
        H = -int(rectangle[1]) + int(rectangle[3])
        padding_H = 0.01 * W
        padding_W = 0.01 * H
        crop_img = my_image[int(rectangle[1]+padding_H):int(rectangle[3]-padding_H), int(rectangle[0]-padding_W):int(rectangle[2]+padding_W)]
#         crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)
        if crop_img is None:
            continue
        if crop_img.shape[0] < 0 or crop_img.shape[1] < 0:
            continue
        cv2.rectangle(draw, (int(rectangle[0]), int(rectangle[1])), (int(rectangle[2]), int(rectangle[3])), (255, 0, 0), 1)

        for i in range(5, 15, 2):
            cv2.circle(draw, (int(rectangle[i + 0]), int(rectangle[i + 1])), 2, (0, 255, 0))


cv2.imwrite("image/out/out.jpg", draw)                       

c = cv2.waitKey(0)

P-Net rectangles shape = (1100, 5)
R-Net rectangles shape = (89, 5)
O-Net rectangles shape = (9, 15)


True